# 1 Introduction

This notebook introduces you a few Matplotlib basic concepts:

1. its design architecture,
2. object hierarchy, and
3. two plotting approaches.

Knowing these basics builds a foundation for you to learn Matplotlib quickly. Navigating and understanding Matplotlib's large online document would be much easier if you have the basic concepts under your belt.

In [ ]:
import matplotlib.pyplot as plt

# "%matplotlib inline" is not needed as inline is the default for notebook environment
# https://github.com/googlecolab/colabtools/issues/616#issuecomment-506829877

import pandas as pd

# for pretty print
from rich import inspect
from rich.pretty import pprint

# 2 Basic Concepts

## 2.1 Architecture

Matplotlib's software architecture has three main layers. From bottom to top, they are backend layer, artist layer, and script layer. We briefly discuss each layer's function.

1. Backend layer (`matplotlib.backend_bases`). Backends handle the hard work behind the scenes to display Matplotlib figures on the screen under different settings/environments. For example, when you plot a figure in a Python shell, a plot window pops up to display the figure. On the other hand, if you use Jupyter notebook (Google Colab included) and draw an inline plot, the plot displays directly in the notebook. The backend layer defines all the primitives that each setting/environment must implement to serve as a backend.

    There are three main components in the backend layer, `FigureCanvasBase`, `RendererBase`, and `event`.

    1.1 `FigureCanvasBase` deals with the area onto which the figure is drawn, i.e., the canvas.

    1.2 `RendererBase` knows how to draw on the `FigureCanvas`, just like a paintbrush.

    1.3 `Event` handles user inputs such as a mouse click.

2. Artist layer (`matplotlib.artist.Artist`). An artist knows how to use a paintbrush (`RendererBase`) to draw on a canvas (`FigureCanvasBase`). All visible elements in a figure are artist objects. Typcially, when composing a plot, we work with artists most of the time. There are two types of artists.

    2.1 Primitives. They are graphical objects we want to paint onto the canvas: `Line2D`, `Rectangle`, `Text`, etc.

    2.2 Containers. They are places to hold the primitives, for example, `Figure`, `Axes`, `Axis`, etc. A container can contain one or more other containers, for example, a `Figure` can contain one or more `Axes` (subplots). `Axes` are Matplotlib's term for subplots (not the plural of Axis). A subplot is a region for plotting. A 2D subplot, for example, contains two `Axis` objects.

3. Script layer (`matplotlib.pyplot`). A light wrapper/interface to the artist layer. It makes plotting simple graphics easy for average users.

As you will see below, whether you work at the artist layer or the script layer represents two different approaches of plotting, but first let us take a look at how a plot organizes its artists objects (primitives and containers).

## 2.2 Object Hierarchy

Matplotlib organizes visual elements (artist objects) of a plot in a hierarchical structure. Sitting at the top of this structure is the `Figure`. A `Figure` can contain multiple `Axes` (subplots) and many other figure-related properties. Inside a `Axes`, there are legends, lines, texts, and many more smaller objects. This hierarchy can be many levels deep.

Below, we write a line of code to create a figure with a single blank subplot (`fig0, ax0 = plt.subplots()`), and drill down the hierarchical structure by following a particular path, from `Figure` at the top to a tick label at the bottom.

```
Figure
 +-- Axes[0]
 |    +-- xaxis
 |    |    +-- MajorTicks[0]
 |    |    |    +-- label1
 |    |    |    +-- ...
 |    |    +-- ...
 |    +-- ...
 +--...
 ```

In [ ]:
# create a figure that contains one subplot/axes
fig0, ax0 = plt.subplots()

# set color for the figure and axes so you know where they are
fig0.set_facecolor('lightskyblue')
ax0.set_facecolor("lightyellow")

In [ ]:
# inspect the figure object
# notice that the figure contains an axes attribute, which is a list of axes
inspect(fig0)

In [ ]:
# double check the axes attribute is a list
pprint(type(fig0.axes))

# the first element of the axes list holds an axes
pprint(type(fig0.axes[0]))

# note that the elemement axes is the same as the one returned by the plt.subplots()
pprint(ax0 is fig0.axes[0])

# inspect the first axes (it's the only axes in this example)
inspect(fig0.axes[0])

In [ ]:
# we can drill down in this fashion: xaxis -> majorTick -> the first tick of the majorTick -> the label of the first tick
pprint(type(fig0.axes[0].xaxis))
pprint(type(fig0.axes[0].xaxis.majorTicks))
pprint(type(fig0.axes[0].xaxis.majorTicks[0]))
pprint(type(fig0.axes[0].xaxis.majorTicks[0].label1))

In [ ]:
# inspect the xaxis
inspect(fig0.axes[0].xaxis)

In [ ]:
# inspect the first tick of the majorTick of xaxis
inspect(fig0.axes[0].xaxis.majorTicks[0])

# inspect the label of the first tick
inspect(fig0.axes[0].xaxis.majorTicks[0].label1)

## 2.3 Two Approaches of plotting

There are two ways to use matplotlib.

1. Work at the script layer. Call `plt.plot()` and other top-level `matplotlib.pyplot` functions to do the plotting (e.g., `plt.bar()`, `plt.title()`, `plt.ylabel()`, etc.) . With this approach you implicitly manage Figures, Axes and other elements in a plot, i.e., you do not specify which Figure and Axes you want to work on, you always work on the *current* Figure and Axes. This approach is sometimes called implicit interface approach.

2. Work at the artist layer. With this approach, you explicitly create Figures and Axes objects and call methods on them. This approach is sometimes called Object Oriented (OO) approach or explicit interface approach.

The first approach is convenient for beginners to get started, however, we recommend using the second approach (so does the official Matplotlib document; see [here](https://matplotlib.org/stable/users/explain/quick_start.html#coding-styles)), particularly for complicated plots, as you can better customize and fine-tune your plots at the artist level. In addition, being explicit on which Figure and Axes you are plotting on is in general a better coding style.

In fact, most of the functions in the first approach is just a thin wrapper of the second approach. For example, the bar plot function `plt.bar()` would simply first get the current Axes instance (`gca()`, get current axes) and then call the `bar()` method of the axes instance. (This is evident if you check the [source code](https://github.com/matplotlib/matplotlib/blob/v3.8.2/lib/matplotlib/pyplot.py#L2728-L2747) of `plt.bar()`)

We demonstrate the two approach below.

In [ ]:
# create a simple dataset for plotting
df = pd.DataFrame(data = {
    'fruits': ['Apple', 'Orange', 'Banana'],
    'sales': [80, 30, 55],
})
df

In [ ]:
# Approach 1 (script layer; implicit; stateful;)

# call plt.bar() function from matplotlib.pyplot
# implicitly create figure and axes
plt.bar(x=df["fruits"], height=df["sales"])

# call pyplot level functions to set title and ylabel
# implicitly refer to the current axes’ title and ylabel
plt.title("Fruit sales")
plt.ylabel("fruit sales (M)")

# show the plot (not needed here in the notebook setting, but needed if running the code in the terminal)
plt.show()

In [ ]:
# Approach 2 (artist layer; explicit; stateless; OOP;)

# create a figure and an axes (subplot)
fig2, ax2 = plt.subplots()

# explicitly call the bar method of the axes just created
ax2.bar(x=df["fruits"], height=df["sales"])

# explicitly call the set_xyz() methods of the axes instance
ax2.set_ylabel("fruit sales (M)")
ax2.set_title("Fruit sales")

# show the plot
plt.show()

When you have multiple subplots, using the second approach (working at the artist layer) allows you to be explicit about which subplot you are working on. This makes the code much more readable.

In [ ]:
# Approach 2 gives cleaner code when having more than one subplot
fig, axes = plt.subplots(nrows=1, ncols=2, layout="constrained")

axes[0].bar(x=df["fruits"], height=df["sales"])
axes[0].set_ylabel("fruit sales (M)")
axes[0].set_title("Fruit Sales")

rects = axes[1].bar(x=df["fruits"], height=df["sales"], color="lightgray")
axes[1].bar_label(rects, padding=3)
axes[1].spines[["top", "right", "left"]].set_visible(False)
axes[1].yaxis.set_visible(False)
axes[1].set_title("Fruit Sales (M)")

fig.suptitle("Default vs Customized")

plt.show()